### Настройки

Начинаем с импорта библиотек

In [4]:
import pandas as pd
import psycopg2

In [5]:
# хороший тон при фиксации изменений в github не публиковать код с открытыми данными для входа
# для запуска кода поместите данные, которые указаны на странице курса в эту ячейку
# при размещении в github удалите эту ячейку
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

Создаем соединение с заданными параметрами

In [6]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

Запрос

In [13]:
# код запроса представляет из себя строковую переменную
query = f'''select a.name, count(distinct e.id) cnt_emp, count(distinct v.id) cnt_vac
            from
                  areas a
                  left join employers e on a.id = e.area
                  left join vacancies v on v.area_id = a.id
            where v.id is null
            group by a.id
            order by cnt_emp desc
         '''
# лучше использовать f-строки - так удобнее передавать параметры

### Выполнение запроса

In [14]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_13568\2473184231.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,cnt_emp,cnt_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


### Финал

In [48]:
# не забываем закрыть соединение после окончания работы
connection.close()